# 학습 조교 에이전트 - (RAG는 가지고 있는 수업자료 ppt 나 pdf로 진행)

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

- 목표
    - 수업자료 기반 Q&A: “PDF에 있는 내용만으로” 정확하고 출처가 분명한 답변.
    - 한국어 최적화: 한글 문장 단위 청크/검색 성능, 용어 통일.
    - 교수자 모드 옵션: “예/복습 퀴즈”, “핵심 요약”, “슬라이드 번호·페이지 인용”까지.

- 기능 모듈(역할별)
    - 질문 답변(QA): 기본 경로. “문서에 없으면 모른다고 말하기” 규칙.
    - 정리 요약: 특정 단원/슬라이드 범위를 요약(목차·키워드·핵심 포인트 5개).
    - 퀴즈 생성: 객관식/단답형/서술형 섞어 난이도 옵션, 정답·근거 포함.
    - 개념 비교: 두 개념 차이점/예시/주의사항 표로 정리.
    - 시험 대비: “출제 가능 포인트” 추출 + 스스로 설명해보기 프롬프트.

In [ ]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime
from langchain.tools import Tool
from langchain.tools.retriever import create_retriever_tool
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


# 1. 문서 로드 및 병합
files = [
    './data/(광통신) chapter 1.pdf',
    './data/(광통신) chapter 2.pdf',
    './data/(광통신) chapter 3.pdf',
    './data/(광통신) chapter 4.pdf'
]

docs = []
for f in files:
    loader = PyMuPDFLoader(f)
    docs.extend(loader.load())

# 필기 필터링
filtered_docs = [doc for doc in docs if len(doc.page_content) > 50]

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(filtered_docs)

# 3. 임베딩 생성 및 벡터 스토어 구축
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 4. 검색기 생성 (retriever)
retriever = vectorstore.as_retriever()

# 5. LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)


# 기능별 도구 정의
# 1. PDF 검색 도구 _ 기존 RAG 체인을 도구로 만듦
pdf_search_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색한다.'
)

# 2. PDF 요약 도구 _ 요약에 특화된 프롬프트와 연결
summary_prompt_template = """
너는 주어진 문서의 내용을 정리본으로 요약하는 전문가야.
오직 주어진 문서의 정보만을 사용해서 요약해줘.
---
문서 내용: {context}
---
"""
summary_prompt = PromptTemplate(template=summary_prompt_template, input_variables=['context'])
summary_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | summary_prompt
    | llm
    | StrOutputParser()
)
pdf_summary_tool = Tool(
    name='pdf_summary',
    func=summary_chain.invoke,
    description='PDF 문서의 내용을 요약한다.'
)

# 3. 퀴즈 생성 프롬프트 (정답 없이 문제만)
quiz_prompt_template = """
너는 주어진 문서 내용으로 퀴즈를 생성하는 전문가야.
객관식 3문제, 단답형 2문제, 서술형 1문제를 만들어줘.

---
문서 내용: {context}
---
"""
quiz_prompt = PromptTemplate(template=quiz_prompt_template, input_variables=['context'])
quiz_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | quiz_prompt
    | llm
    | StrOutputParser()
)
# 퀴즈 생성 도구
pdf_quiz_tool = Tool(
    name='pdf_quiz',
    func=quiz_chain.invoke,
    description='주어진 PDF 문서의 내용으로 퀴즈를 생성한다.'
)

# 4. 정답 생성 프롬프트 (채점 시 호출됨)
quiz_answer_prompt_template = """
너는 퀴즈 채점 전문가야.
아래 문서 내용을 바탕으로, 이전에 출제한 퀴즈 문제에 대한 정답과 근거를 작성해줘.
각 문제에 대해 정답과 간단한 근거를 제공해줘. 페이지 정보도 포함해줘.

---
문서 내용: {context}
---
[문제]
{quiz_questions}
"""
quiz_answer_prompt = PromptTemplate(template=quiz_answer_prompt_template, input_variables=['context', 'quiz_questions'])
quiz_answer_chain = (
    {'context': retriever, 'quiz_questions': RunnablePassthrough()}
    | quiz_answer_prompt
    | llm
    | StrOutputParser()
)

# 5. 채점 프롬프트
grader_prompt_template = """
너는 퀴즈 채점 전문가야. 아래에 주어진 퀴즈의 정답과 사용자의 답변을 비교하여 채점하고, 각 문제에 대한 상세한 피드백과 해설을 제공해줘.

[정답]
{quiz_answers}

[사용자 답변]
{user_answers}

채점 결과와 피드백 및 해설을 작성해줘.
"""
grader_prompt = PromptTemplate(template=grader_prompt_template, input_variables=['quiz_answers', 'user_answers'])
grader_chain = (
    grader_prompt
    | llm
    | StrOutputParser()
)

# 6. 퀴즈 채점 도구
def grade_quiz_with_memory(user_input):
    quiz_full_output = None
    # 대화 기록을 역순으로 탐색하며 가장 최근의 퀴즈 문제 메시지를 찾습니다.
    # LLM의 답변 메시지 중 '퀴즈를 내드립니다.' 같은 메시지를 찾습니다.
    for message in reversed(memory.load_memory_variables({})['chat_history']):
        if '퀴즈를 내드립니다' in message.content:
            quiz_full_output = message.content
            break

    if quiz_full_output is None:
        return "죄송합니다. 퀴즈 문제가 저장되어 있지 않습니다. 먼저 퀴즈를 생성해주세요."
    
    # 정답 생성 체인을 호출하여 정답을 동적으로 생성
    quiz_answers = quiz_answer_chain.invoke(quiz_full_output)

    chain_input = {
        'quiz_answers': quiz_answers,
        'user_answers': user_input
    }
    
    return grader_chain.invoke(chain_input)

pdf_quiz_grader = Tool(
    name='pdf_quiz_grader',
    func=grade_quiz_with_memory,
    description='사용자의 퀴즈 답변을 채점하고 상세한 피드백 및 해설을 제공한다.'
)

# 7. 기타 도구들
compare_prompt_template = """
너는 두 개념을 비교하는 전문가야. 아래 문서 내용을 바탕으로 다음 두 개념을 비교하여 차이점, 공통점, 그리고 예시를 표 형태로 정리해줘.
---
문서 내용: {context}
---
개념1: {concept1}
개념2: {concept2}
"""
compare_prompt = PromptTemplate(template=compare_prompt_template, input_variables=['context', 'concept1', 'concept2'])
compare_chain = (
    {'context': retriever, 'concept1': RunnablePassthrough(), 'concept2': RunnablePassthrough()}
    | compare_prompt
    | llm
    | StrOutputParser()
)
pdf_compare_tool = Tool(
    name='pdf_compare',
    func=compare_chain.invoke,
    description='두 개의 개념을 비교하여 차이점, 공통점, 예시를 표로 정리한다.'
)

exam_prep_prompt_template = """
너는 시험 출제 전문가야. 주어진 문서 내용에서 시험에 나올 만한 핵심 포인트 5가지를 추출하고, 각 포인트에 대한 설명을 직접 해볼 수 있도록 정리해줘.
---
문서 내용: {context}
---
"""
exam_prep_prompt = PromptTemplate(template=exam_prep_prompt_template, input_variables=['context'])
exam_prep_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | exam_prep_prompt
    | llm
    | StrOutputParser()
)
pdf_exam_prep_tool = Tool(
    name='pdf_exam_prep',
    func=exam_prep_chain.invoke,
    description='시험 출제 가능성이 높은 핵심 포인트 5가지를 추출하고 설명해준다.'
)

# 8. 도구 리스트
tools = [
    pdf_search_tool,
    pdf_summary_tool,
    pdf_quiz_tool,
    pdf_quiz_grader,
    pdf_compare_tool,
    pdf_exam_prep_tool
]

# 9. 에이전트 구성
today = datetime.today().strftime('%Y년 %m월 %d일')
system_message = f"""
너는 광통신 시스템 수업 자료를 기반으로 한 학습 조교 에이전트야.
사용자의 질문에 답하기 위해 주어진 도구들을 사용해.

- 만약 사용자가 '요약'이나 '정리'를 요청하면, 'pdf_summary' 도구를 사용해.
- 만약 사용자가 '퀴즈'나 '문제'를 요청하면 'pdf_quiz' 도구를 사용해.
- 만약 사용자가 '채점'이라는 키워드와 함께 답변을 제공하면, 'pdf_quiz_grader' 도구를 사용해.
- 만약 사용자가 '비교'를 요청하면, 'pdf_compare' 도구를 사용해.
- 만약 사용자가 '시험'이나 '출제'를 요청하면, 'pdf_exam_prep' 도구를 사용해.
- 그 외의 질문에 대해서는 'pdf_search' 도구를 사용해서 답변을 찾아.
- 답변은 반드시 주어진 PDF 문서의 내용에만 근거해야 하고 어떤 페이지에 있는지까지 알려줘야 해.
- 문서에 없는 내용은 '제공된 문서에서 관련 정보를 찾을 수 없습니다.'라고 답해.

오늘은 {today}이야.
"""
prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory,
    tools=tools,
    verbose=True
)

In [129]:
agent_executor.invoke({'input': '퀴즈 내줘.'})



> Entering new AgentExecutor chain...

Invoking: `pdf_quiz` with `광통신 시스템`


객관식 문제 1:
광통신에서 사용하는 광섬유의 특징으로 옳지 않은 것은 무엇인가요?
a) 많은 양의 데이터를 빠르게 전송할 수 있다  
b) 파장이 짧을수록 직진성이 강하다  
c) 광섬유는 항상 제거할 수 없다  

정답: c) 광섬유는 항상 제거할 수 없다

객관식 문제 2:
다음 중 광통신에서 사용하는 변조 방식에 대한 설명으로 옳은 것은 무엇인가요?
a) 가우시안 형태의 광신호는 변조에 적합하다  
b) 광신호는 항상 가우시안 형태를 띄어야 한다  
c) 광신호는 변조기 없이도 전송이 가능하다  

정답: a) 가우시안 형태의 광신호는 변조에 적합하다

객관식 문제 3:
광통신에서 TDM(시간 분할 다중화)의 주요 특징으로 옳지 않은 것은 무엇인가요?
a) 여러 고객이 하나의 채널을 시간 단위로 나누어 사용한다  
b) 주파수 차이를 이용하여 여러 신호를 동시에 전송한다  
c) 주파수 할당이 클수록 더 많은 고객이 사용할 수 있다  

정답: b) 주파수 차이를 이용하여 여러 신호를 동시에 전송한다

단답형 문제 1:
광통신에서 사용하는 파장의 범위는 무엇인가요?  

답: 800 nm ~ 6695 nm

단답형 문제 2:
광섬유의 파장이 짧을수록 어떤 특성이 강해지나요?  

답: 직진성

서술형 문제 1:
광통신에서 광섬유의 분산이 왜 중요한지 설명하시오.이제 퀴즈를 제공해드렸습니다. 추가로 더 필요하시면 말씀해 주세요!

> Finished chain.


{'input': '퀴즈 내줘.',
 'chat_history': [HumanMessage(content='퀴즈 내줘.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='이제 퀴즈를 제공해드렸습니다. 추가로 더 필요하시면 말씀해 주세요!', additional_kwargs={}, response_metadata={})],
 'output': '이제 퀴즈를 제공해드렸습니다. 추가로 더 필요하시면 말씀해 주세요!'}

In [130]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime
from langchain.tools import Tool
from langchain.tools.retriever import create_retriever_tool
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


# 1. 문서 로드 및 병합
files = [
    './data/(광통신) chapter 1.pdf',
    './data/(광통신) chapter 2.pdf',
    './data/(광통신) chapter 3.pdf',
    './data/(광통신) chapter 4.pdf'
]

docs = []
for f in files:
    loader = PyMuPDFLoader(f)
    docs.extend(loader.load())

# 필기 필터링
filtered_docs = [doc for doc in docs if len(doc.page_content) > 50]

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(filtered_docs)

# 3. 임베딩 생성 및 벡터 스토어 구축
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 4. 검색기 생성 (retriever)
retriever = vectorstore.as_retriever()

# 5. LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)


# 기능별 도구 정의
# 1. PDF 검색 도구 _ 기존 RAG 체인을 도구로 만듦
pdf_search_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색한다.'
)

# 2. PDF 요약 도구 _ 요약에 특화된 프롬프트와 연결
summary_prompt_template = """
너는 주어진 문서의 내용을 정리본으로 요약하는 전문가야.
오직 주어진 문서의 정보만을 사용해서 요약해줘.
---
문서 내용: {context}
---
"""
summary_prompt = PromptTemplate(template=summary_prompt_template, input_variables=['context'])
summary_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | summary_prompt
    | llm
    | StrOutputParser()
)
pdf_summary_tool = Tool(
    name='pdf_summary',
    func=summary_chain.invoke,
    description='PDF 문서의 내용을 요약한다.'
)

# 3. 퀴즈 생성 프롬프트 (정답과 해설 함께 생성)
quiz_prompt_template = """
너는 주어진 문서 내용으로 퀴즈를 생성하는 전문가야.
객관식 3문제, 단답형 2문제, 서술형 1문제를 만들어줘.

[문제]
1. 광통신 시스템의 세 가지 주요 구성 요소는 무엇인가요?
2. ...

[정답_및_근거]
1. 정답: 광원, 광섬유, 광검출기 (근거: 광통신 시스템은 광원, 광섬유, 광검출기로 구성된다. 출처: [페이지 2])
2. 정답: ...

퀴즈를 낸 후, 사용자에게 '답변을 입력하고 채점이라고 말하면 채점해드립니다' 라고 말해줘.
---
문서 내용: {context}
---
"""
quiz_prompt = PromptTemplate(template=quiz_prompt_template, input_variables=['context'])
quiz_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | quiz_prompt
    | llm
    | StrOutputParser()
)
pdf_quiz_tool = Tool(
    name='pdf_quiz',
    func=quiz_chain.invoke,
    description='주어진 PDF 문서의 내용으로 퀴즈를 생성한다.'
)

# 4. 퀴즈 채점 도구
grader_prompt_template = """
너는 퀴즈 채점 전문가야. 아래에 주어진 퀴즈의 정답과 사용자의 답변을 비교하여 채점하고, 각 문제에 대한 상세한 피드백과 해설을 제공해줘.

정답과 해설에는 반드시 해당 정답의 근거가 되는 문서의 페이지 번호를 포함해줘.
예시:
Q1. 문제 내용
A1. 정답 내용 (근거: ~~~, 출처: [페이지 번호] )

[정답]
{quiz_answers}

[사용자 답변]
{user_answers}

채점 결과와 피드백 및 해설을 작성해줘.
"""
grader_prompt = PromptTemplate(template=grader_prompt_template, input_variables=['quiz_answers', 'user_answers'])

def grade_quiz_with_memory(user_input):
    quiz_full_output = None
    # 대화 기록을 역순으로 탐색하며 퀴즈 정답이 포함된 메시지를 찾습니다.
    # 이 방식은 [-2] 인덱스보다 훨씬 안정적입니다.
    for message in reversed(memory.load_memory_variables({})['chat_history']):
        if '[정답_및_근거]' in message.content:
            quiz_full_output = message.content
            break
    
    if quiz_full_output is None:
        return "죄송합니다. 채점을 위한 퀴즈 정답을 찾을 수 없습니다. 퀴즈를 다시 내주시면 채점해드리겠습니다."
    
    try:
        quiz_answers = quiz_full_output.split('[정답_및_근거]')[1].strip()
    except IndexError:
        return "죄송합니다. 퀴즈 형식에 문제가 있어 정답을 추출할 수 없습니다."
    
    chain_input = {
        'quiz_answers': quiz_answers,
        'user_answers': user_input
    }
    
    return grader_chain.invoke(chain_input)

grader_chain = (
    grader_prompt
    | llm
    | StrOutputParser()
)

pdf_quiz_grader = Tool(
    name='pdf_quiz_grader',
    func=grade_quiz_with_memory,
    description='사용자의 퀴즈 답변을 채점하고 상세한 피드백 및 해설을 제공한다.'
)

# 5. 개념 비교 도구
compare_prompt_template = """
너는 두 개념을 비교하는 전문가야. 아래 문서 내용을 바탕으로 다음 두 개념을 비교하여 차이점, 공통점, 그리고 예시를 표 형태로 정리해줘.
---
문서 내용: {context}
---
개념1: {concept1}
개념2: {concept2}
"""
compare_prompt = PromptTemplate(template=compare_prompt_template, input_variables=['context', 'concept1', 'concept2'])
compare_chain = (
    {'context': retriever, 'concept1': RunnablePassthrough(), 'concept2': RunnablePassthrough()}
    | compare_prompt
    | llm
    | StrOutputParser()
)
pdf_compare_tool = Tool(
    name='pdf_compare',
    func=compare_chain.invoke,
    description='두 개의 개념을 비교하여 차이점, 공통점, 예시를 표로 정리한다.'
)

# 6. 시험 대비 도구
exam_prep_prompt_template = """
너는 시험 출제 전문가야. 주어진 문서 내용에서 시험에 나올 만한 핵심 포인트 5가지를 추출하고, 각 포인트에 대한 설명을 직접 해볼 수 있도록 정리해줘.
---
문서 내용: {context}
---
"""
exam_prep_prompt = PromptTemplate(template=exam_prep_prompt_template, input_variables=['context'])
exam_prep_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | exam_prep_prompt
    | llm
    | StrOutputParser()
)
pdf_exam_prep_tool = Tool(
    name='pdf_exam_prep',
    func=exam_prep_chain.invoke,
    description='시험 출제 가능성이 높은 핵심 포인트 5가지를 추출하고 설명해준다.'
)

tools = [pdf_search_tool, pdf_summary_tool, pdf_quiz_tool, pdf_quiz_grader, pdf_compare_tool, pdf_exam_prep_tool]

# 에이전트 구축하기
today = datetime.today().strftime('%Y년 %m월 %d일')
system_message = f"""
너는 광통신 시스템 수업 자료를 기반으로 한 학습 조교 에이전트야.
사용자의 질문에 답하기 위해 주어진 도구들을 사용해.

- 만약 사용자가 '요약'이나 '정리'를 요청하면, 'pdf_summary' 도구를 사용해.
- 만약 사용자가 '퀴즈'나 '문제'를 요청하면 'pdf_quiz' 도구를 사용해.
- 만약 사용자가 '채점'이라는 키워드와 함께 답변을 제공하면, 'pdf_quiz_grader' 도구를 사용해.
- 만약 사용자가 '비교'를 요청하면, 'pdf_compare' 도구를 사용해.
- 만약 사용자가 '시험'이나 '출제'를 요청하면, 'pdf_exam_prep' 도구를 사용해.
- 그 외의 질문에 대해서는 'pdf_search' 도구를 사용해서 답변을 찾아.
- 답변은 반드시 주어진 PDF 문서의 내용에만 근거해야 하고 어떤 페이지에 있는지까지 알려줘야 해.
- 문서에 없는 내용은 '제공된 문서에서 관련 정보를 찾을 수 없습니다.'라고 답해.

오늘은 {today}이야.
"""
prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory,
    tools=tools,
    verbose=True
)

In [132]:
agent_executor.invoke({'input': '1번 답 a, 2번 답 b, 3번 답 c 채점해줘'})



> Entering new AgentExecutor chain...

Invoking: `pdf_quiz_grader` with `1번 답 a, 2번 답 b, 3번 답 c`


죄송합니다. 채점을 위한 퀴즈 정답을 찾을 수 없습니다. 퀴즈를 다시 내주시면 채점해드리겠습니다.
Invoking: `pdf_quiz_grader` with `1번 답 a, 2번 답 b, 3번 답 c`


죄송합니다. 채점을 위한 퀴즈 정답을 찾을 수 없습니다. 퀴즈를 다시 내주시면 채점해드리겠습니다.채점 결과, 답변이 정답이 아니거나 채점할 수 없는 답변입니다. 정답을 다시 확인하시거나, 정답을 입력해 주세요.

> Finished chain.


{'input': '1번 답 a, 2번 답 b, 3번 답 c 채점해줘',
 'chat_history': [HumanMessage(content='퀴즈 내줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='퀴즈를 생성하였습니다. 답변을 입력하시거나 채점 원하시면 말씀해 주세요.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='1번 답 a, 2번 답 b, 3번 답 c 채점해줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='채점 결과, 답변이 정답이 아니거나 채점할 수 없는 답변입니다. 정답을 다시 확인하시거나, 정답을 입력해 주세요.', additional_kwargs={}, response_metadata={})],
 'output': '채점 결과, 답변이 정답이 아니거나 채점할 수 없는 답변입니다. 정답을 다시 확인하시거나, 정답을 입력해 주세요.'}

In [133]:
import os
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain import hub
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from datetime import datetime
from langchain.tools import Tool
from langchain.tools.retriever import create_retriever_tool
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
from operator import itemgetter


# 1. 문서 로드 및 병합
files = [
    './data/(광통신) chapter 1.pdf',
    './data/(광통신) chapter 2.pdf',
    './data/(광통신) chapter 3.pdf',
    './data/(광통신) chapter 4.pdf'
]

docs = []
for f in files:
    loader = PyMuPDFLoader(f)
    docs.extend(loader.load())

# 필기 필터링
filtered_docs = [doc for doc in docs if len(doc.page_content) > 50]

# 2. 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(filtered_docs)

# 3. 임베딩 생성 및 벡터 스토어 구축
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

# 4. 검색기 생성 (retriever)
retriever = vectorstore.as_retriever()

# 5. LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano', temperature=0)

# 퀴즈 저장용 변수 (전역)
current_quiz_data = {
    'questions': None,
    'context': None
}

# 기능별 도구 정의
# 1. PDF 검색 도구 _ 기존 RAG 체인을 도구로 만듦
pdf_search_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색한다.'
)

# 2. PDF 요약 도구 _ 요약에 특화된 프롬프트와 연결
summary_prompt_template = """
너는 주어진 문서의 내용을 정리본으로 요약하는 전문가야.
오직 주어진 문서의 정보만을 사용해서 요약해줘.
---
문서 내용: {context}
---
"""
summary_prompt = PromptTemplate(template=summary_prompt_template, input_variables=['context'])
summary_chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | summary_prompt
    | llm
    | StrOutputParser()
)
pdf_summary_tool = Tool(
    name='pdf_summary',
    func=summary_chain.invoke,
    description='PDF 문서의 내용을 요약한다.'
)

# 3. 수정된 퀴즈 생성 프롬프트 (정답과 해설 절대 포함하지 않음)
quiz_prompt_template = """
너는 주어진 문서 내용으로 퀴즈를 생성하는 전문가야.
다음 규칙을 반드시 지켜서 문제만 출제해줘:

1. 객관식 3문제, 단답형 2문제, 서술형 1문제를 만들어줘.
2. 절대로 정답이나 해설을 포함하지 마.
3. 오직 문제만 출제해.
4. 각 문제 번호를 명확히 표시해.

---
문서 내용: {context}
---

퀴즈를 내드립니다:
"""

def generate_quiz(question):
    # 관련 문서 검색
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    # 퀴즈 생성
    quiz_prompt = PromptTemplate(template=quiz_prompt_template, input_variables=['context'])
    quiz_chain = quiz_prompt | llm | StrOutputParser()
    
    quiz_questions = quiz_chain.invoke({'context': context})
    
    # 현재 퀴즈 정보 저장 (채점용)
    current_quiz_data['questions'] = quiz_questions
    current_quiz_data['context'] = context
    
    return quiz_questions

pdf_quiz_tool = Tool(
    name='pdf_quiz',
    func=generate_quiz,
    description='주어진 PDF 문서의 내용으로 퀴즈를 생성한다. 문제만 출제하고 정답은 포함하지 않는다.'
)

# 4. 정답 생성 프롬프트 (채점 시 호출됨)
quiz_answer_prompt_template = """
너는 퀴즈 채점 전문가야.
아래 문서 내용을 바탕으로, 다음 퀴즈 문제에 대한 정답과 상세한 해설을 작성해줘.
각 문제에 대해 정확한 정답과 근거가 되는 페이지 정보도 포함해줘.

---
문서 내용: {context}
---

[출제된 문제]
{quiz_questions}

정답 및 해설:
"""

def generate_quiz_answers():
    if current_quiz_data['questions'] is None or current_quiz_data['context'] is None:
        return "저장된 퀴즈 정보가 없습니다."
    
    quiz_answer_prompt = PromptTemplate(
        template=quiz_answer_prompt_template, 
        input_variables=['context', 'quiz_questions']
    )
    quiz_answer_chain = quiz_answer_prompt | llm | StrOutputParser()
    
    return quiz_answer_chain.invoke({
        'context': current_quiz_data['context'],
        'quiz_questions': current_quiz_data['questions']
    })

# 5. 수정된 채점 프롬프트
grader_prompt_template = """
너는 퀴즈 채점 전문가야. 
다음과 같이 채점하고 피드백을 제공해줘:

1. 먼저 정답을 제시해
2. 사용자의 답변을 각 문제별로 채점해
3. 틀린 문제에 대해서는 상세한 해설과 올바른 답을 제공해
4. 맞은 문제에 대해서도 간단한 확인을 해줘
5. 전체 점수를 계산해줘

[모범답안 및 해설]
{quiz_answers}

[사용자 답변]
{user_answers}

채점 결과:
"""

def grade_quiz_function(user_input):
    if current_quiz_data['questions'] is None:
        return "죄송합니다. 채점할 퀴즈 문제가 없습니다. 먼저 퀴즈를 생성해주세요."
    
    # 정답 생성
    quiz_answers = generate_quiz_answers()
    
    # 채점
    grader_prompt = PromptTemplate(
        template=grader_prompt_template, 
        input_variables=['quiz_answers', 'user_answers']
    )
    grader_chain = grader_prompt | llm | StrOutputParser()
    
    result = grader_chain.invoke({
        'quiz_answers': quiz_answers,
        'user_answers': user_input
    })
    
    return result

pdf_quiz_grader = Tool(
    name='pdf_quiz_grader',
    func=grade_quiz_function,
    description='사용자의 퀴즈 답변을 채점하고 정답과 상세한 해설을 제공한다.'
)

# 7. 기타 도구들
compare_prompt_template = """
너는 두 개념을 비교하는 전문가야. 아래 문서 내용을 바탕으로 다음 두 개념을 비교하여 차이점, 공통점, 그리고 예시를 표 형태로 정리해줘.
---
문서 내용: {context}
---
개념1: {concept1}
개념2: {concept2}
"""

def compare_concepts(input_text):
    # 입력에서 두 개념 추출 (간단한 파싱)
    concepts = input_text.replace('비교', '').replace('해줘', '').strip().split('과', 1)
    if len(concepts) != 2:
        concepts = input_text.replace('비교', '').replace('해줘', '').strip().split('와', 1)
    
    if len(concepts) != 2:
        return "두 개념을 '과' 또는 '와'로 구분해서 입력해주세요. 예: 'A와 B 비교해줘'"
    
    concept1 = concepts[0].strip()
    concept2 = concepts[1].strip()
    
    # 문서 검색
    retrieved_docs = retriever.invoke(f"{concept1} {concept2}")
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    compare_prompt = PromptTemplate(
        template=compare_prompt_template, 
        input_variables=['context', 'concept1', 'concept2']
    )
    compare_chain = compare_prompt | llm | StrOutputParser()
    
    return compare_chain.invoke({
        'context': context,
        'concept1': concept1,
        'concept2': concept2
    })

pdf_compare_tool = Tool(
    name='pdf_compare',
    func=compare_concepts,
    description='두 개의 개념을 비교하여 차이점, 공통점, 예시를 표로 정리한다.'
)

exam_prep_prompt_template = """
너는 시험 출제 전문가야. 주어진 문서 내용에서 시험에 나올 만한 핵심 포인트 5가지를 추출하고, 각 포인트에 대한 설명을 직접 해볼 수 있도록 정리해줘.
---
문서 내용: {context}
---
"""

def prepare_exam(question):
    retrieved_docs = retriever.invoke(question)
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    exam_prep_prompt = PromptTemplate(template=exam_prep_prompt_template, input_variables=['context'])
    exam_prep_chain = exam_prep_prompt | llm | StrOutputParser()
    
    return exam_prep_chain.invoke({'context': context})

pdf_exam_prep_tool = Tool(
    name='pdf_exam_prep',
    func=prepare_exam,
    description='시험 출제 가능성이 높은 핵심 포인트 5가지를 추출하고 설명해준다.'
)

# 8. 도구 리스트
tools = [
    pdf_search_tool,
    pdf_summary_tool,
    pdf_quiz_tool,
    pdf_quiz_grader,
    pdf_compare_tool,
    pdf_exam_prep_tool
]

# 9. 수정된 에이전트 구성
today = datetime.today().strftime('%Y년 %m월 %d일')
system_message = f"""
너는 광통신 시스템 수업 자료를 기반으로 한 학습 조교 에이전트야.
사용자의 질문에 답하기 위해 주어진 도구들을 정확히 사용해.

도구 사용 규칙:
- '요약', '정리' 요청 → 'pdf_summary' 도구 사용
- '퀴즈', '문제' 요청 → 'pdf_quiz' 도구 사용 (문제만 출제, 정답 포함 금지)
- '채점', '답안 확인', '정답 알려줘' + 사용자 답변 → 'pdf_quiz_grader' 도구 사용
- '비교' 요청 → 'pdf_compare' 도구 사용  
- '시험', '출제', '핵심 포인트' 요청 → 'pdf_exam_prep' 도구 사용
- 그 외 질문 → 'pdf_search' 도구 사용

중요한 규칙:
1. 퀴즈를 낼 때는 절대로 정답이나 해설을 포함하지 마라.
2. 사용자가 채점을 요청할 때만 정답과 해설을 제공해라.
3. 답변은 반드시 주어진 PDF 문서 내용에만 근거해야 해.
4. 페이지 정보도 함께 제공해.
5. 문서에 없는 내용은 '제공된 문서에서 관련 정보를 찾을 수 없습니다'라고 답해.

오늘은 {today}이야.
"""

prompt = ChatPromptTemplate.from_messages([
    ('system', system_message),
    MessagesPlaceholder(variable_name='chat_history'),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad')
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt
)

agent_executor = AgentExecutor(
    agent=agent,
    memory=memory,
    tools=tools,
    verbose=True
)



In [144]:
agent_executor.invoke({'input': '그럼 WDM이랑 TDM 비교 분석해서 정리해줘.'})



> Entering new AgentExecutor chain...

Invoking: `pdf_compare` with `WDM과 TDM의 차이점, 공통점, 그리고 각각의 예시를 표로 정리해 주세요.`


| 구분             | WDM (Wavelength Division Multiplexing)                         | TDM (Time Division Multiplexing)                                |
|------------------|--------------------------------------------------------------|--------------------------------------------------------------|
| 개념             | 여러 파장(빛의 파장)을 나누어 동시에 여러 신호 전송             | 시간 슬롯을 나누어 하나의 채널에서 여러 신호를 순차적으로 전송  |
| 차이점           | - 파장(빛의 색깔)을 이용하여 다중화<br>- 병렬 전송 방식<br>- 많은 양의 데이터 전송 가능 | - 시간(슬롯)을 나누어 다중화<br>- 순차 전송 방식<br>- 적은 대역폭 사용 가능 |
| 공통점           | - 둘 다 다중화 기법으로, 하나의 통신 채널에서 여러 신호를 전송<br>- 광통신 및 유선 통신에서 사용 | - 둘 다 여러 사용자 또는 신호를 하나의 매체를 통해 전송하는 기술<br>- 데이터 전송 효율 향상 목적 |
| 예시             | - 광섬유 통신에서 여러 파장을 동시에 사용하는 경우<br>- 대용량 데이터 전송에 적합 | - 디지털 전화 시스템에서 시간 슬롯을 나누어 여러 통화 연결<br>- 디지털 신호 처리에서 시간 분할 다중화 활용 |
| 특징             | - 높은 대역폭과 빠른 전송 속도 가능<br>- 파장 간 간섭(분산) 문제 고려 필요 | - 간단

{'input': '그럼 WDM이랑 TDM 비교 분석해서 정리해줘.',
 'chat_history': [HumanMessage(content='퀴즈 내줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='다음은 광통신 시스템에 관한 퀴즈입니다. 정답은 포함되어 있지 않으며, 문제만 출제되었습니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='1번 답은 a인 것 같아. 채점해줘.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='전체 채점 결과, 4점 만점에 4점을 받으셨습니다! 1번 문제는 틀리셨지만, 나머지 문제들은 모두 정답입니다. 계속해서 학습에 힘쓰시면 좋겠습니다!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='그럼 시험 대비 해서 어떻게 공부해야할지 알려줘.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='광통신 시스템에 대한 시험 대비 핵심 포인트 5가지와 설명을 정리해 드렸습니다. 이를 바탕으로 각 포인트를 이해하고, 관련 개념을 정리하는 것이 좋습니다. 추가로 궁금한 점이 있거나, 특정 포인트에 대해 더 자세한 설명이 필요하시면 말씀해 주세요!', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='이거랑 관련해서 문제를 또 내줄 수 있어 ?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='이 문제들은 광통신 시스템의 핵심 개념과 관련된 내용입니다. 더 많은 문제를 원하시거나, 특정 주제에 대한 문제를 원하시면 말씀해 주세요!', 